# Análisis geográfico

In [3]:
import pandas as pd
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime 
import dateutil
import folium
from folium import plugins

A partir del archivo <i>Cinemas.csv</i> marcar en el mapa cada Cine existente diferenciando con color aquellos que tengas página web de los que no la tengan.

In [78]:


df = pd.read_csv('data/Cinemas.csv')

df_coor = df[['website','longitude','latitude']]



cinemas_map = folium.Map(location=[df_coor['latitude'][0],df_coor['longitude'][0]], zoom_start=10)

df['website'] = df['website'].astype(str).str.split()
df_coor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   website    12 non-null     object 
 1   longitude  116 non-null    float64
 2   latitude   116 non-null    float64
dtypes: float64(2), object(1)
memory usage: 2.8+ KB


In [95]:
def addMarket(row):
    
    if row['website'] == 'nan':
        market = folium.Marker(location=[float(row['latitude']), float(row['longitude'])], icon=folium.Icon(color="red",icon_color="black")).add_to(cinemas_map)
    else:
        folium.Marker(location=[float(row['latitude']), float(row['longitude'])],radius=10).add_to(cinemas_map)

    
    

df_coor.astype(str).apply(lambda row:addMarket(row),axis=1)
cinemas_map

<div style="display: flex;">
	<div style="width: 25px; height: 25px; border-radius: 50px; background-color: red;">
	</div>
	<div style="padding-left: 15px;">
		Cines con Sitio Web
	</div>	
</div>

<div style="display: flex; margin-top: 15px;">
	<div style="width: 25px; height: 25px; border-radius: 50px; background-color: #2880ca;">
	</div>
	<div style="padding-left: 15px;">
		Cines sin Sitio Web
	</div>	
</div>

A partir del archivo <i>fires.csv</i> marcar en el mapa con un mapa de calor los sectores donde han ocurrido incendios forestales en Argentina.

In [47]:
df = pd.read_csv('data/fires.csv', decimal=".")

df_fires = df[['longitude','latitude']]
df_fires = df_fires.dropna()


fires_map = folium.Map(location=[df_fires['latitude'][0],df_fires['longitude'][0]], zoom_start=1)
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,-32.909,-65.624,303.6,1.4,1.2,2020-10-11,330,Terra,MODIS,30,6.0NRT,282.5,12.0,N
1,-32.927,-65.604,319.6,1.4,1.2,2020-10-11,330,Terra,MODIS,56,6.0NRT,283.0,29.8,N
2,-32.937,-65.602,333.4,1.4,1.2,2020-10-11,330,Terra,MODIS,100,6.0NRT,284.7,57.3,N
3,-33.237,-65.849,309.4,1.3,1.1,2020-10-11,330,Terra,MODIS,38,6.0NRT,279.0,17.6,N
4,-33.235,-65.835,342.7,1.3,1.1,2020-10-11,330,Terra,MODIS,100,6.0NRT,282.3,76.5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5160,-24.476,-63.417,338.8,3.4,1.7,2020-10-25,1725,Aqua,MODIS,83,6.0NRT,303.7,146.2,D
5161,-24.482,-63.424,355.1,3.4,1.7,2020-10-25,1725,Aqua,MODIS,97,6.0NRT,303.6,341.2,D
5162,-24.276,-63.777,327.1,3.6,1.8,2020-10-25,1725,Aqua,MODIS,34,6.0NRT,302.5,52.9,D
5163,-23.977,-63.709,323.7,3.6,1.8,2020-10-25,1725,Aqua,MODIS,55,6.0NRT,304.0,51.5,D


In [50]:
from folium.plugins import HeatMap
values = df_fires.values

fires_map.add_child(plugins.HeatMap(values, radius=10))
fires_map

In [123]:
def addMarket(row):
    folium.Marker(location=[float(row['latitude']), float(row['longitude'])],radius=10).add_to(fires_map)
        

df_fires.astype(str).apply(lambda row:addMarket(row),axis=1)
fires_map

A partir del archivo <i>incendios_forestales.csv</i> marcar con un círculo en el mapa cada incendio forestal modificando el tamaño de cada círculo dependiendo de la cantidad de hectareas afectadas.
Al pasar el mouse por encima de cada círculo mostrar la fecha del incendio.

In [45]:
df = pd.read_csv('data/incendios-forestales.csv',sep=",", encoding='cp1252')

df_incendios = df[['current_size','fire_location_latitude','fire_location_longitude','fire_year']]
df_incendios['current_size'] = df_incendios['current_size']*100

incendios_map = folium.Map(location=[df_fires['latitude'][0],df_fires['longitude'][0]], zoom_start=5)
df_incendios

<ipython-input-45-824c1af79f01>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incendios['current_size'] = df_incendios['current_size']*100


,current_size,fire_location_latitude,fire_location_longitude,fire_year
0,20.0,51.152933,-115.034600,2006
1,1.0,51.157633,-115.002133,2006
2,1.0,51.194400,-114.516167,2006
3,75.0,51.125617,-114.841683,2006
4,1.0,50.409833,-114.478967,2006
...,...,...,...,...
19839,1.0,54.626783,-115.598067,2018
19840,1.0,54.350250,-115.083683,2018
19841,1.0,54.022550,-115.668667,2018
19842,1.0,54.023100,-115.669533,2018


In [ ]:
def addCircleMarker(row):
    circle = folium.CircleMarker(location=[float(row['fire_location_latitude']), float(row['fire_location_longitude'])],radius=float(row['current_size']), popup=row['fire_year'])
    circle.add_to(incendios_map)    

df_incendios.apply(lambda row:addCircleMarker(row),axis=1)
incendios_map